This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [1]:
# get api key from your .env file
import os
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')
print(len(API_KEY))

20


Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests
import collections

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [3]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
r = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json',data={'start_date':'2020-07-14', 'end_date':'2020-07-14',
                'api_key':API_KEY})

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
r.json()

{'dataset_data': {'limit': None,
  'transform': None,
  'column_index': None,
  'column_names': ['Date',
   'Open',
   'High',
   'Low',
   'Close',
   'Change',
   'Traded Volume',
   'Turnover',
   'Last Price of the Day',
   'Daily Traded Units',
   'Daily Turnover'],
  'start_date': '2020-07-14',
  'end_date': '2020-07-14',
  'frequency': 'daily',
  'data': [['2020-07-14',
    90.1,
    90.15,
    88.1,
    89.2,
    None,
    183507.0,
    16350975.75,
    None,
    None,
    None]],
  'collapse': None,
  'order': None}}

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).

In [5]:
r = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json',data={'api_key':API_KEY,
                'start_date':'2017-01-01',
                'end_date':'2017-12-31'})

2. Convert the returned JSON object into a Python dictionary.


In [6]:
ds = dict(r.json())

3. Calculate what the highest and lowest opening prices were for the stock in this period.


In [7]:
data = ds['dataset_data']['data']
openings = [row[1] for row in data if row[1]!= None]
print(f'The highest opening price was ${round(max(openings),2)} and the lowest opening price was ${round(min(openings),2)}')

The highest opening price was $53.11 and the lowest opening price was $34.0


4. What was the largest change in any one day (based on High and Low price)?


In [8]:
max_dif = max([row[2]-row[3] for row in data])
print(f'The largest change in any one day between the high and low prices was ${round(max_dif,2)}')

The largest change in any one day between the high and low prices was $2.81


5. What was the largest change between any two days (based on Closing Price)?


In [9]:
two_day_dif = 0

for i, row in enumerate(data):
    if i > 0:
        dif = abs(row[4]-data[i-1][4])
        if dif>two_day_dif:
            two_day_dif=dif

print(f'The largest change between any two days based on closing price is ${round(two_day_dif,2)}')


The largest change between any two days based on closing price is $2.56


6. What was the average daily trading volume during this year?

In [10]:
daily_vol = [row[6] for row in data]
avg_daily_vol = sum(daily_vol)/len(daily_vol)
print(f'The average daily trading volume during 2017 was {avg_daily_vol}')

The average daily trading volume during 2017 was 89124.33725490196


7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [12]:
def find_median(l):
    l = sorted(l)
    if len(l)%2 != 0:
        #if the length of the list is an odd number then use the middle number
        #take floor of the list len divided by two
        return l[len(l)//2]
    else:
        #if length of list is an even number median must be calced from both middle values
        
        #the first middle index
        o = len(l)//2
        #the second middle index
        t = o-1
        #return the mean of the values at the indices 
        return (l[o]+l[t])/2

print(f'The median trading volume during 2017 was {find_median(daily_vol)}')



The median trading volume during 2017 was 76286.0
